# Combine all PDF files into single file

In [ ]:
# import os
# os.environ["TOKENIZERS_PARALLELISM"] = "false"
# os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [ ]:
# !ollama list
# !ollama pull gemma3:12b
#!ollama rm gemma3:4b

In [ ]:
import os

pdf_dir = '/Users/avosseler/Business Trips/2025/Barcelona' 
os.listdir(pdf_dir)

In [ ]:
from finance_analysis.utils.utils import merge_pdfs

merge_pdfs(pdf_dir, pdf_names=["flight.pdf", "car_sharing_first_day.pdf"], output_file='merged.pdf')

In [ ]:
# Check out chunker library
# uv add chonkie

In [ ]:
from PyPDF2 import PdfMerger
import os

# Set the directory containing your PDFs
os.chdir(pdf_dir)

# Create a PdfMerger object
merger = PdfMerger()
#print(os.listdir(pdf_dir))

first_file = "Travel Expense Barcelona.pdf"

merger.merge(0, first_file)
print(f'Adding {first_file}...')

# Loop through all PDF files in the directory and append them
for page_number, file in enumerate(sorted(os.listdir(".")), start=1):
    if file.endswith('.pdf') and file != first_file:
        print(f'Appending {file}...')
        # merger.append(file)
        merger.merge(page_number, file)


# Write out the merged PDF
merger.write('combined_invoice.pdf')
merger.close()
print('Merged PDF created as combined PDF')

### Chat

In [ ]:
import os

file_dir = '/Users/avosseler/Business Trips/2025/Barcelona'
file_name = 'Hotel_Taxi.pdf'

with open(os.path.join(file_dir, file_name), 'rb') as f:
    image_bytes = f.read()

In [ ]:
from pdf2image import convert_from_path
import os

file_dir = '/Users/avosseler/Business Trips/2025/Barcelona'
file_name = 'Hotel_Taxi.pdf'

# Convert PDF to images
images = convert_from_path(os.path.join(file_dir, file_name))

# Save the first page as an image
image_path = os.path.join(file_dir, 'Hotel_Taxi_Page1.png')
images[0].save(image_path, 'PNG')
print(f"Saved first page as image: {image_path}")

In [ ]:
import ollama

# Read the image as bytes
with open(image_path, 'rb') as f:
    image_bytes = f.read()

prompt = "Extract all text from this image and present it in Markdown format."
prompt = "What is the name of the guest in the hotel invoice? Extract the name."

# Send the image to the Ollama model
response = ollama.chat(
    model='gemma3:12b',
    messages=[{
        'role': 'user',
        'content': prompt,
        'images': [image_bytes]
    }]
)

print(response.message.content)

### LangChain

In [ ]:
import base64
from PIL import Image
from io import BytesIO

def encode_image_to_base64(image_path):
    with Image.open(image_path) as img:
        buffered = BytesIO()
        img.save(buffered, format="PNG")  # or "PNG" if appropriate
        return base64.b64encode(buffered.getvalue()).decode("utf-8")


In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage

image_path = os.path.join(file_dir, 'Hotel_Taxi_Page1.png')

# Encode the image
image_b64 = encode_image_to_base64(image_path=image_path)

query = "Extract all text from this image and present it in Markdown format."
query = "What is the name of the guest in the hotel invoice?"
query = "What is the total amount in Euro on the invoice? You will find the final amount at the end of the invoice."
query = "What is the arrival date of the guest?"
query = "What is the departure date of the guest?"


content_parts = [
    {
        "type": "image_url",
        "image_url": f"data:image/png;base64,{image_b64}",
    },
    {
        "type": "text",
        "text": query
    }
]

message = HumanMessage(content=content_parts)

llm = ChatOllama(model="gemma3:12b", temperature=0.2, max_tokens=2000)

response = llm.invoke([message])
print(response.content)


## Docling OCR

In [ ]:
from finance_analysis.resources.document_processor import DocumentProcessor
import os

file_name = 'Hotel_Taxi.pdf'
#file_name = "taxi_to_airport_barc.pdf"
#file_name = "car_sharing_first_day.pdf"
#file_name = "flight.pdf"

file_path = os.path.join('/Users/avosseler/Business Trips/2025/Barcelona', file_name)

dproc = DocumentProcessor(file_path=file_path)
document = dproc.process()

In [ ]:
# document.texts
# document.tables

In [ ]:
#dproc.display_markdown()

markdown = document.export_to_markdown()

In [ ]:
from finance_analysis.resources.invoice_classifier import InvoiceDetector

clf = InvoiceDetector()

await clf.adetect(input_text=markdown)

Docling

In [5]:
import os
import warnings
from IPython.display import Markdown, display
from docling.document_converter import DocumentConverter

#file_name = 'Taxi_from_Budapest_Airport_to_Hotel_Day1.pdf'
file_name = "Taxi_from_Budapest_Airport_to_Hotel_BookingCom.png"
folder_name = 'Graphisoft-Budapest-Aug/Taxi'

file_path = os.path.join('/Users/avosseler/Business Trips/2025', folder_name, file_name)

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    
    # Your existing Docling code
    converter = DocumentConverter()
    result = converter.convert(
        file_path,     
    )    
    markdown = result.document.export_to_markdown()
    
display(Markdown(markdown))

<!-- image -->

<!-- image -->

<!-- image -->

<!-- image -->

<!-- image -->

<!-- image -->

<!-- image -->

Buchungsinformationen

Details zur Abholung

Abgeschlossen Letzte Aktualisierung: 31 Aug. um 12:32 Uhr

## Ihre Buchungsinformationen

Bestätigungsnummer: 144119682

## Reisedetails

<!-- image -->

Flugverfolgung hinzugefügt

Ihre Abholzeit richtet sich nach der Ankunftszeit Ihres Flugs

Der Fahrer wird Ihren Flug im Auge behalten und die Abholzeit im Falle von Verspätungen anpassen. Nach seiner Ankunft wartet der Fahrer 45 Minuten, sodass Sie Zeit haben; vom Flugzeug zum Treffpunkt zu gelangen.

## Flugnummer

LH1678

## Angaben zu Abholung und Rückgabe

Mo. 25. Aug-

Budapest Liszt Ferenc International Airport (BUD) , Budapest, 1185 Ungarn

38 Min

Mo. 25. Aug. (ungefähr)

Four Points by Sheraton Budapest Danube, Budapest, Pozsonyi út 79, 1133 Ungarn

Standard

Bereitgestellt von Hungary Private Transfers

Fahrgastauswahl

Maximal Fahrgast

Kontaktdaten

Alexander Vosseler +4917621942264 a.vosseler@gmx.net

## Preisübersicht

Einzelfahrt

30,21 €

Gesamtpreis

30,21 €

Einschließlich aller Steuern und Gebühren

## Wie können wir Ihnen helfen?

Zu den Hilfeseiten

Kundenservice ist rund um die Uhr verfügbar

## Benötigen Sie eine Rückfahrt?

<!-- image -->

<!-- image -->

Suche

<!-- image -->

<!-- image -->

Gespeichert

Buchungen

<!-- image -->

In [6]:
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions, TesseractOcrOptions
from docling.document_converter import DocumentConverter, PdfFormatOption

pipeline_options = PdfPipelineOptions()
pipeline_options.do_ocr = True
pipeline_options.ocr_options = TesseractOcrOptions(lang=["eng"])  # or use other OCR option classes

converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(
            pipeline_options=pipeline_options
        )
    }
)

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')

    result = converter.convert(file_path)
    
markdown = result.document.export_to_markdown()
display(Markdown(markdown))


<!-- image -->

<!-- image -->

<!-- image -->

<!-- image -->

<!-- image -->

<!-- image -->

<!-- image -->

Buchungsinformationen

Details zur Abholung

Abgeschlossen Letzte Aktualisierung: 31 Aug. um 12:32 Uhr

## Ihre Buchungsinformationen

Bestätigungsnummer: 144119682

## Reisedetails

<!-- image -->

Flugverfolgung hinzugefügt

Ihre Abholzeit richtet sich nach der Ankunftszeit Ihres Flugs

Der Fahrer wird Ihren Flug im Auge behalten und die Abholzeit im Falle von Verspätungen anpassen. Nach seiner Ankunft wartet der Fahrer 45 Minuten, sodass Sie Zeit haben; vom Flugzeug zum Treffpunkt zu gelangen.

## Flugnummer

LH1678

## Angaben zu Abholung und Rückgabe

Mo. 25. Aug-

Budapest Liszt Ferenc International Airport (BUD) , Budapest, 1185 Ungarn

38 Min

Mo. 25. Aug. (ungefähr)

Four Points by Sheraton Budapest Danube, Budapest, Pozsonyi út 79, 1133 Ungarn

Standard

Bereitgestellt von Hungary Private Transfers

Fahrgastauswahl

Maximal Fahrgast

Kontaktdaten

Alexander Vosseler +4917621942264 a.vosseler@gmx.net

## Preisübersicht

Einzelfahrt

30,21 €

Gesamtpreis

30,21 €

Einschließlich aller Steuern und Gebühren

## Wie können wir Ihnen helfen?

Zu den Hilfeseiten

Kundenservice ist rund um die Uhr verfügbar

## Benötigen Sie eine Rückfahrt?

<!-- image -->

<!-- image -->

Suche

<!-- image -->

<!-- image -->

Gespeichert

Buchungen

<!-- image -->

In [ ]:
from langchain_google_vertexai import VertexAIEmbeddings, ChatVertexAI
from finance_analysis.config import  global_config as glob

embeddings = VertexAIEmbeddings(
    model_name="text-embedding-005",
    project=glob.GCP_PROJECT
)

llm = ChatVertexAI(
    model_name="gemini-2.5-flash-preview-04-17",
    temperature=0.2,
    project=glob.GCP_PROJECT,
)

Extract infos from Taxi invoice

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

class OutputStructure(BaseModel):
    total_amount: str = Field(..., description="The total amount in Euro on the invoice. Use the format as 1234.56")
    issue_date: str = Field(..., description="The issue date of the invoice. Use the date format as DD.MM.YYYY")

parser = JsonOutputParser(pydantic_object=OutputStructure)

taxi_prompt = PromptTemplate(
    template="""
    You are an intelligent assistant tasked with answering questions based on the provided context. 
    Use only the information in the context to answer the questions accurately. Provide the answers in JSON format.

    Context:
    {context}

    Questions:
    1. What is the total amount in Euro on the invoice? You will find the final amount at the end of the invoice.
    2. What is the date of issue?

    Please use the following output format:
    {format_instructions}
    """,
    input_variables=["context"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = taxi_prompt | llm | parser

response = chain.invoke({"context": markdown})
response

Extract Infos from Hotel Invoice

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

class OutputStructure(BaseModel):
    guest_name: str = Field(..., description="The name of the guest in the hotel invoice. Use the format as 'Firstname Lastname'")
    total_amount: str = Field(..., description="The total amount in Euro on the invoice. Use the format as 1234.56")
    start_date: str = Field(..., description="The arrival date of the guest. Use the date format as DD.MM.YYYY")
    end_date: str = Field(..., description="The departure date of the guest. Use the date format as DD.MM.YYYY")

parser = JsonOutputParser(pydantic_object=OutputStructure)

hotel_prompt = PromptTemplate(
    template="""
    You are an intelligent assistant tasked with answering questions based on the provided context. 
    Use only the information in the context to answer the questions accurately. Provide the answers in JSON format.

    Context:
    {context}

    Questions:
    1. What is the name of the guest in the hotel invoice?
    2. What is the total amount in Euro on the invoice? You will find the final amount at the end of the invoice.
    3. What is the arrival date of the guest?
    4. What is the departure date of the guest?

    Please use the following output format:
    {format_instructions}
    """,
    input_variables=["context"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = hotel_prompt | llm | parser

response = chain.invoke({"context": markdown})
response

In [ ]:
from finance_analysis.resources.document_processor import DocumentProcessor
import os

# file_name = 'Hotel_Taxi.pdf'
# file_name = "taxi_to_airport_barc.pdf"
# file_name = "car_sharing_first_day.pdf"
file_name = "flight.pdf"

file_path = os.path.join('/Users/avosseler/Business Trips/2025/Barcelona', file_name)

# step 1: Process the document
dproc = DocumentProcessor(file_path=file_path)
document = dproc.process()
markdown = document.export_to_markdown()

# step 2: Extract entities
from finance_analysis.resources.extractor import EntityExtractor

ent = EntityExtractor("flight")

extracted = await ent.extract_entities(markdown)
extracted

In [ ]:
extracted["total_amount"]

In [ ]:
# from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
# from langchain_core.documents import Document

# headers_to_split_on = [
#     ("#", "Header 1"),
#     ("##", "Header 2"),
#     ("###", "Header 3"),
# ]

# markdown_splitter = MarkdownHeaderTextSplitter(
#     headers_to_split_on=headers_to_split_on,
#     strip_headers=False
# )

# header_chunks = markdown_splitter.split_text(markdown)
# len(header_chunks)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

chunks = text_splitter.create_documents([markdown])
chunks

# from langchain_text_splitters import CharacterTextSplitter

# text_splitter = CharacterTextSplitter(
#     chunk_size=1000,
#     chunk_overlap=200
# )

# chunks = text_splitter.create_documents([markdown])


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

vector_store = FAISS.from_documents(chunks, embeddings)

retriever = vector_store.as_retriever(search_kwargs={"k": 5})

prompt_template = """
Answer using only this context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(prompt_template)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

query = "What is the name of the guest in the hotel invoice?"
query = "What is the total amount in Euro on the invoice? You will find the final amount at the end of the invoice."
query = "What is the arrival date of the guest?"
query = "What is the departure date of the guest?"

response = qa_chain.invoke({
    "query": query
})
print(response["result"])


### Via Docling Loader

In [ ]:
from docling.chunking import HybridChunker
from langchain_docling import DoclingLoader
from langchain_docling.loader import ExportType
import os

# Your existing Docling conversion
file_dir = '/Users/avosseler/Business Trips/2025/Barcelona'
file_name = 'Hotel_Taxi.pdf'
file_path = os.path.join(file_dir, file_name)

EMBED_MODEL_ID = "sentence-transformers/all-MiniLM-L6-v2"

loader = DoclingLoader(
    file_path=file_path,
    export_type=ExportType.DOC_CHUNKS,
    chunker=HybridChunker(tokenizer=EMBED_MODEL_ID),
)

docs = loader.load()

In [ ]:
docs

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

vector_store = FAISS.from_documents(docs, embeddings)

retriever = vector_store.as_retriever(search_kwargs={"k": 5})

prompt_template = """
Answer using only this context:
{context}

Question: {question}
"""

prompt = PromptTemplate.from_template(prompt_template)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:
query = "What is the name of the guest in the hotel invoice? Extract the name."
query = "What is the total amount due on the invoice?"

response = qa_chain.invoke({
    "query": query
})

print(response["result"])

# LangGraph 

In [ ]:
from finance_analysis.resources.document_processor import DocumentProcessor
import os

file_name = 'Hotel_Taxi.pdf'
# file_name = "taxi_to_airport_barc.pdf"
# file_name = "car_sharing_first_day.pdf"
#file_name = "flight.pdf"

file_path = os.path.join('/Users/avosseler/Business Trips/2025/Barcelona', file_name)

# step 1: Process the document
dproc = DocumentProcessor(file_path=file_path)
document = dproc.process()
markdown = document.export_to_markdown()

# step 2: Extract entities
from finance_analysis.resources.extractor import EntityExtractor

ent = EntityExtractor("flight")

extracted = ent.extract_entities(markdown)
extracted

In [ ]:
from finance_analysis.resources.invoice_classifier import InvoiceDetector

clf = InvoiceDetector()

result = await clf.adetect(input_text=markdown)
result

### Singe file:

In [ ]:
import os
from typing import TypedDict, Annotated, Literal
from operator import add
from langgraph.graph import StateGraph, START, END
from finance_analysis.resources.extractor import EntityExtractor
from finance_analysis.resources.document_processor import DocumentProcessor
from finance_analysis.resources.invoice_classifier import InvoiceDetector


# 1) Define the graph state
class DocState(TypedDict):
    file_name: str
    processed_doc: str
    invoice_type: str  
    entities: Annotated[list[dict[str, str]], add]    # must be a list for concatenation


# 2) Document-processing node 
def process_document(state: DocState) -> dict[str, str]:
    print("Processing document...")    
    file_path = os.path.join('/Users/avosseler/Business Trips/2025/Barcelona', state["file_name"])

    # step 1: Process the document
    dproc = DocumentProcessor(file_path=file_path)
    document = dproc.process()
    processed = document.export_to_markdown()

    return {"processed_doc": processed}

# 3) Entity-extraction node 
async def extract_entities(state: DocState) -> dict[str, list[dict[str, str]]]:
    print("Extracting entities...")
    
    ent = EntityExtractor(state["invoice_type"])

    extracted = await ent.aextract_entities(state["processed_doc"])
    # print(extracted)
    return {"entities": [extracted]}

# 4) Invoice classification node 
async def classify_invoice(state: DocState) -> dict[str, str]:
    print("Classifying invoice...")
    clf = InvoiceDetector()

    result = await clf.adetect(input_text=state["processed_doc"])
    print(result)
    return {"invoice_type": result['invoice_type']}

# Conditional routing after processing
def should_classify(state: DocState) -> Literal["classify", "end"]:
    if len(state["processed_doc"]) > 0:
        return "classify"
    return "end"

# Conditional routing after classification
def should_extract(state: DocState) -> Literal["extract", "end"]:
    if state.get("invoice_type"):
        return "extract"
    return "end"


# 5) Build the graph
builder = StateGraph(DocState)
builder.add_node("process", process_document)
builder.add_node("classify", classify_invoice)
builder.add_node("extract", extract_entities)
builder.add_edge(START, "process")

# Set static edges (vs dynamically added edges via Command commands)

# After processing, classify if possible
builder.add_conditional_edges(
    "process",
    should_classify,
    {
        "classify": "classify",
        "end": END
    }
)

# After classification, extract entities if possible
builder.add_conditional_edges(
    "classify",
    should_extract,
    {
        "extract": "extract",     # If should_extract returns "extract", the workflow moves to the "extract" node
        "end": END                # If should_extract returns "end", the workflow terminates (END is a special constant)
    }
)

# Connect extraction to end
builder.add_edge("extract", END)

# Compile graph
processor_graph = builder.compile()

In [ ]:
# from IPython.display import Image, display

# display(Image(processor_graph.get_graph().draw_mermaid_png()))

In [ ]:
file_name = 'Hotel_Taxi.pdf'
file_name = "taxi_to_airport_barc.pdf"
file_name = "car_sharing_first_day.pdf"
file_name = "flight.pdf"

result = await processor_graph.ainvoke({"file_name": file_name})
result

### Loop over list of names. Use dynamic routing/edges via Command

In [ ]:
import os
import warnings
from typing import TypedDict, Annotated, Dict
from operator import add
import requests
from langgraph.graph import END
from langgraph.types import Command
#from langchain_core.tools import tool
from langchain.prompts import PromptTemplate
from langchain_google_vertexai import ChatVertexAI
from finance_analysis.utils.prompts import summary_prompt
from finance_analysis.config import  global_config as glob
from finance_analysis.utils.utils import create_conversion_info

from finance_analysis.resources.extractor import EntityExtractor
from finance_analysis.resources.document_processor import DocumentProcessor
from finance_analysis.resources.invoice_classifier import InvoiceDetector
from finance_analysis.utils.data_models import CurrencyCodeLiteral
from finance_analysis.utils.utils import update_travel_expense_xlsx

warnings.filterwarnings('ignore')

directory = "Graphisoft, Budapest, Mar"
#directory = "Barcelona"
root_directory = '/Users/avosseler/Business Trips/2025'
target_dir = os.path.join(root_directory, directory)


class DocState(TypedDict):
    file_names: list[str]  # List of files to process
    current_file_index: int     # Index of the current file
    file_name: str         # Name of the current file
    processed_doc: str
    invoice_type: str
    entities: Annotated[list[dict[str, str]], add]
    inferred_types: Annotated[list[str], add]      # collect all inferred types
    currencies: Annotated[list[CurrencyCodeLiteral], add]  # collect all currencies
    descriptions: Annotated[list[str], add]       # describe each invoice, Hotel Name, etc.
    summary: str  # Summary of the processed documents
    rate_info: str  # Information about exchange rates
    
# Load the next file node
def load_next_file(state: DocState) -> Command:
    index = state.get("current_file_index", 0)
    file_list = state.get("file_names", [])
    
    # Finally: Goto Summarization if no more files
    if index >= len(file_list):
        return Command(goto="summarize")
    
    print(f"\nLoading file: {file_list[index]}")
    return Command(
        update={
            "file_name": file_list[index],
            "current_file_index": index + 1
        },
        goto="process"          # next step (-> edge)
    )

# Document-processing node 
def process_document(state: DocState) -> Command:
    print("Processing document...")    
    file_path = os.path.join(target_dir, state["file_name"])

    dproc = DocumentProcessor(file_path=file_path)
    document = dproc.process()
    processed = document.export_to_markdown()
    return Command(update={"processed_doc": processed}, goto="classify")


async def extract_and_convert(state: DocState) -> Command:
    
    # pull raw entities
    extracted = await EntityExtractor(state["invoice_type"])\
                           .aextract_entities(state["processed_doc"])
                           
    # Convert immediately
    amount = float(extracted["total_amount"])
    from_cur = extracted["currency"]
    
    print(f"Converting {amount} {from_cur} to EUR...")
    conv = convert_currency(amount, from_cur)
    print(f"Converted amount: {conv['EUR Amount']} EUR")
    
    extracted["total_amount"] = conv["EUR Amount"]
    extracted["currency"] = "EUR"
    extracted["invoice_type"] = state["invoice_type"]

    # stash both original + conversion, then loop back
    return Command(
        update={
            "entities": [extracted],
            "conversion_result": conv,
            "currencies": [from_cur],      # must be list for concatenation, as you use the 'add' operator above
            "descriptions": [extracted.get("description", "No description provided")]
        },
        goto="load"
    )

# Invoice classification node 
async def classify_invoice(state: DocState) -> Command:
    
    print("Classifying invoice...")
    clf = InvoiceDetector()
    result = await clf.adetect(input_text=state["processed_doc"])
    
    print(result['invoice_type'])
    
    return Command(
        update={
            "invoice_type": result['invoice_type'],
            "inferred_types": [result['invoice_type']]
        },
        goto="extract"
    )
    
# @tool
# def sum_calculator(numbers: list[int]) -> int:
#     """
#     A simple tool to calculate the sum of a list of numbers.

#     Args:
#         numbers (list[int]): A list of integers to sum up.

#     Returns:
#         int: The total sum of the numbers in the list.
#     """
#     if not isinstance(numbers, list):
#         raise ValueError("Input should be a list of integers.")
#     if not all(isinstance(num, int) for num in numbers):
#         raise ValueError("All elements in the list should be integers.")
#     return sum(numbers)


#@tool("convert_currency", return_direct=True)
def convert_currency(
    amount: float,
    from_currency: CurrencyCodeLiteral,
) -> Dict[str, str]:
    """
    Convert a monetary amount from one currency to another via api.frankfurter.app

    Args:
        amount (float): Input amount to convert
        from_currency (CurrencyCodeLiteral): Abbreviation of the currency to convert from, e.g. "USD"

    Returns:
        Dict[str, str]: A dictionary containing the converted amount and the date of conversion
    """
    if from_currency == "EUR":
        return {"EUR Amount": amount, "Exchange Rate - Date": "N/A"}
    
    # If currency is not Euro, convert to Euro:
    resp = requests.get(
        "https://api.frankfurter.app/latest",
        params={"amount": f"{amount}", "from": f"{from_currency.upper()}", "to": "EUR"},
        timeout=5, verify=False     # no ssl verification
    )
    resp.raise_for_status()
    euro_amount = resp.json()["rates"].get("EUR")     # Converted amount in EUR
    date = resp.json()["date"]
    print(resp.json())
    return {"EUR Amount": euro_amount, "Exchange Rate - Date": date}


async def summarize(state: DocState) -> Command:
    """
    Generates a summary of extracted entities from the given document state.

    This method logs the summarization process, ensures that entities are present in the state,
    creates exchange rate information, and uses a prompt template with a language model to
    generate a summary. The summary is then returned as part of a Command object to update the
    state and proceed to the end of the workflow.

    Args:
        state (DocState): The current document state containing extracted entities and other relevant data.

    Returns:
        Command: A command object containing the generated summary and the next workflow step.
    """

    assert "entities" in state, "No entities found in the result."

    rate_info = create_conversion_info(state)

    llm = ChatVertexAI(
        model_name="gemini-2.5-flash",
        temperature=0.1,
        project=glob.GCP_PROJECT,
    )

    summary_templ = PromptTemplate(
        template=summary_prompt,
        input_variables=["context", "info_exchange_rate"],
    )

    chain = summary_templ | llm

    response = await chain.ainvoke(
        {"context": state["entities"], "info_exchange_rate": rate_info}
    )
    
    return Command(
        update={
            "summary": response.content,
            "rate_info": rate_info,
        },
        # goto=END,
        goto="update_xlsx"  # Proceed to the next step
    )
    
    
def update_xlsx_file(state: DocState) -> Command:
    """
    Placeholder for a method to edit an XLSX file based on the current state.
    This method is not implemented yet.

    Args:
        state (DocState): The current processing state.

    Returns:
        Command: A command to proceed to the next step in the workflow.
    """
    update_travel_expense_xlsx(result=state)
    print("Updating XLSX file with the extracted data...")
    return Command(goto=END)

In [ ]:
from langgraph.graph import StateGraph, START
# from langgraph.prebuilt import ToolNode

builder = StateGraph(DocState)

# tool_node = ToolNode([sum_calculator])
# builder.add_node("calculate sum", tool_node)

# # turn your function into a graph node
# builder.add_node("convert", ToolNode([convert_currency]))

builder.add_node("load", load_next_file)
builder.add_node("process", process_document)
builder.add_node("classify", classify_invoice)
builder.add_node("extract", extract_and_convert)
builder.add_node("summarize", summarize)
builder.add_node("update_xlsx", update_xlsx_file)
builder.add_edge(START, "load")

# This is only statically added here to see the edges in the visualization
# the graph will be compiled with the edges added dynamically through the use of 'Command'
# REMOVE THIS LATER: will cause recursive loop otherwise and an error!

# builder.add_edge("load", "process")
# builder.add_edge("process", "classify")
# builder.add_edge("classify", "extract")
# builder.add_edge("extract", "load").     # loop back to load next file
# builder.add_edge("load", "summarize")   # finally go to summarize
# builder.add_edge("summarize", "update_xlsx")
# builder.add_edge("update_xlsx", END)


# Compile the graph
processor_graph = builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(processor_graph.get_graph().draw_mermaid_png()))

In [ ]:
# list_of_files = [
#         "Hotel_Taxi.pdf",
#         "taxi_to_airport_barc.pdf",
#         "car_sharing_first_day.pdf",
#         "flight.pdf"
#     ]

list_of_files = [
        "Bolt to GS 2.pdf",
        # "Bolt to GS.pdf",
        "Hotel.pdf",
        # "Hotel_Taxi.pdf",
    ]

In [ ]:
initial_state = {
    "file_names": list_of_files,
    "current_index": 0,
    "entities": [],
    "inferred_types": []
}

result = await processor_graph.ainvoke(initial_state)

# Add the inferred types to the entities for later use
for i,typ in enumerate(result["inferred_types"]):
    result["entities"][i]["invoice_type"] = typ

In [ ]:
result

Create Markdown summary

In [ ]:
from IPython.display import Markdown, display

display(Markdown(result["summary"]))

### Wrapped:

In [ ]:
import os
from finance_analysis.resources.agent import ProcessorGraph

directory = "Graphisoft, Budapest, Mar"
# directory = "Barcelona"
root_directory = '/Users/avosseler/Business Trips/2025'
target_dir = os.path.join(root_directory, directory)
target_xlsx_file = "my_travel_expense.xlsx"

# list_of_files = [
#         "Hotel_Taxi.pdf",
#         "taxi_to_airport_barc.pdf",
#         "car_sharing_first_day.pdf",
#         "flight.pdf"
#     ]

list_of_files = [
        # "Bolt to GS 2.pdf",
        "Bolt to GS.pdf",
        "Hotel.pdf",
        # "Hotel_Taxi.pdf",
    ]

supervisor = ProcessorGraph(list_of_files=list_of_files, source_path=target_dir, target_xls_file=target_xlsx_file)

In [ ]:
result = await supervisor.ainvoke()

In [ ]:
result

In [ ]:
from IPython.display import Markdown, display

display(Markdown(result["summary"]))

In [ ]:
result["entities"]#[0]

### Update XLS file

In [ ]:
import os
from openpyxl import load_workbook
from openpyxl.workbook.properties import CalcProperties

dir_name = "/Users/avosseler/Business Trips/2025/tmp"
pfile = os.path.join(dir_name, 'Travel Expense Tmp.xlsx')

# Load your workbook (make sure data_only=False so formulas are preserved)
wb = load_workbook(pfile, data_only=False)

# Select the sheet you want to modify
ws = wb['RKA Seite 1']

# Overwrite the cells that feed into formulas
ws['C2'] = "Vosseler, Alexander"
ws['E2'] = "100392"  # cost center
ws['E3'] = "Munich"                # location
ws['C6'] = "Barcelona"             # destination
ws['C7'] = "Workshop"          # Reason for travel

rowA = 19; entries = 1; rowB = 29
for res in result["entities"]:
    if res['invoice_type'] == "hotel":
        ws[f"A{rowA}"] = res['checkin_date']
        ws[f"B{rowA}"] = entries
        ws[f"C{rowA}"] = res['description']
        ws[f"E{rowA}"] = res['total_amount']
        rowA += 1
    else:
        ws[f"A{rowB}"] = res['issue_date']
        ws[f"B{rowB}"] = entries
        ws[f"C{rowB}"] = res['description']
        ws[f"E{rowB}"] = res['total_amount']
        rowB += 1
    entries += 1
    
ws[f"C{rowB+2}"] = result["rate_info"]  # Add exchange rate info
        
# Manually attach CalcProperties to force full recalc on load
wb._calculation_properties = CalcProperties(fullCalcOnLoad=True)

# Save to a new file (or overwrite)
wb.save(os.path.join(dir_name, 'Travel Expense Tmp Edt2.xlsx'))


In [ ]:
ws2 = wb['RKA Seite 2']

In [ ]:
import os
from openpyxl import load_workbook
from openpyxl.workbook.properties import CalcProperties

def update_travel_expense_xlsx(
    result,
    trip_metadata={"Last/First name": "Vosseler, Alexander", 
                   "Location": "Munich", 
                   "Destination": "Barcelona", 
                   "Cost Center": "100392",
                   "Reason for travel": "Workshop"},
    dir_name="/Users/avosseler/Business Trips/2025/tmp",
    input_file="Travel Expense Tmp.xlsx",
    output_file="Travel Expense Tmp Edt.xlsx",
):
    """
    Update the travel expense Excel file with extracted invoice data.

    Args:
        result (dict): The result dictionary containing invoice entities.
        dir_name (str): Directory containing the Excel file.
        input_file (str): Name of the input Excel file.
        output_file (str): Name of the output Excel file.
        sheet_name (str): Name of the worksheet to update.
    """
    pfile = os.path.join(dir_name, input_file)
    wb = load_workbook(pfile, data_only=False)
    ws = wb["RKA Seite 1"]
    
    # --------------------------- Sheet 1 ------------------------------------
    # Overwrite the cells that feed into formulas
    ws['C2'] = trip_metadata.get("Last/First name", "Vosseler, Alexander")
    ws['E2'] = trip_metadata.get("Cost Center", "100392")  # cost center
    ws['E3'] = trip_metadata.get("Location", "Munich")  # location
    ws['C6'] = trip_metadata.get("Destination", "Barcelona")  # destination
    ws['C7'] = trip_metadata.get("Reason for travel", "Workshop")  # Reason for travel

    rowA = 19
    entries = 1
    rowB = 29
    for res in result["entities"]:
        if res['invoice_type'] == "hotel":
            ws[f"A{rowA}"] = res['checkin_date']
            ws[f"B{rowA}"] = entries
            ws[f"C{rowA}"] = res['description']
            ws[f"E{rowA}"] = res['total_amount']
            rowA += 1
        else:
            ws[f"A{rowB}"] = res['issue_date']
            ws[f"B{rowB}"] = entries
            ws[f"C{rowB}"] = res['description']
            ws[f"E{rowB}"] = res['total_amount']
            rowB += 1
        entries += 1

    ws[f"C{rowB+2}"] = result["rate_info"]  # Add exchange rate info
    
    # --------------------------- Sheet 2 ------------------------------------
    ws2 = wb['RKA Seite 2']

    # Manually attach CalcProperties to force full recalc on load
    wb._calculation_properties = CalcProperties(fullCalcOnLoad=True)

    # Save to a new file (or overwrite)
    wb.save(os.path.join(dir_name, output_file))

In [ ]:
update_travel_expense_xlsx(
    result,
    trip_metadata={
        "Last/First name": "Vosseler, Alexander",
        "Location": "Munich",
        "Destination": "Barcelona",
        "Reason for travel": "Workshop"
    },
    dir_name="/Users/avosseler/Business Trips/2025/tmp",
    input_file="Travel Expense Tmp.xlsx",
    output_file="Travel Expense Tmp Edt3.xlsx"
)